In [ ]:
# set the flag to NOT run the tests
doRunTests = False
# run the outside notebook (to get the functions)
%run -i "checkFilesInVisit.ipynb"


visits = [
    "/Users/denismottet/Documents/GitHub/Rearm-CheckFiles/dat/ReArm.lnk/ReArm_C1P022", 
    "/Users/denismottet/Documents/GitHub/Rearm-CheckFiles/dat/ReArm.lnk/ReArm_C1P023",
]

# check the files in the visit
for pathToData in visits:
    if not os.path.isdir(pathToData):
        print("ERROR: pathToData is not a directory: ", pathToData)
        continue
    print("Checking visit: ", pathToData)
    vistToCheck = CheckFilesInRearmVisit(pathToData)
    vistToCheck.checkFilesByVisit()
    vistToCheck.printProblems()
    # vistToCheck.printGoodFiles()
    vistToCheck.saveCheckResults()